In [ ]:
#
# Author: Manny Alvarez
#
# Network and Complexity 
# 
# Downey Chapter 5 Exercises 
# 
#


In [ ]:
### Packages 
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
        
download('https://github.com/AllenDowney/ThinkComplexity2/raw/master/notebooks/utils.py')
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import seaborn as sns

from utils import decorate, savefig

# Set the random seed so the notebook 
# produces the same results every time.
np.random.seed(17)


In [ ]:
################ 5.1

'''
Write a version of correlate that returns the same result as
np.correlate with mode='same'. Hint: use the NumPy function pad.
'''
## numpy.correlate(a, v, mode='same') ###
import numpy as np

## Use cross correlation function from 5.11 section in the book 

def c_k(a, w, k):
    """Compute element k of the cross correlation of a and w.
    """
    N = len(w)
    return sum(a[k:k+N] * w)

#######################


np.correlate([1, 2, 3], [0, 1, 0.5], 'same') # from numpy documentation 

def manny_correlate(a,v):
    values= len(a)
    paddarows = np.pad(a,1,'constant') # pad the row with zeros for size
    crsscorr = [c_k(paddarows, v, i) for i in range(values)]
    return np.array(crsscorr)


manny_correlate([1, 2, 3], [0, 1, 0.5])      
 


In [ ]:
############## 5.2  ########
'''

This exercise asks you to experiment with Rule 110 and some
of its spaceships.
1. Read the Wikipedia page about Rule 110, which describes its background
pattern and spaceships: http://thinkcomplex.com/r110.
2. Create a Rule 110 CA with an initial condition that yields the stable
background pattern.
Note that the Cell1D class provides start_string, which allows you to
initialize the state of the array using a string of 1s and 0s.
3. Modify the initial condition by adding different patterns in the center
of the row and see which ones yield spaceships. You might want to
enumerate all possible patterns of n bits, for some reasonable value of
n. For each spaceship, can you find the period and rate of translation?
What is the biggest spaceship you can find?
4. What happens when spaceships collide?

'''

'''

 Rule 110, is (1) turing complete whihc means  it can computate any function, also knonw as universality. Rules in 110 refers to a specific set of rules where the new cell is 
    dependednt on the current value (much like all CAs) but in this case the rule is summarized by 01101110. As for spacships, these refer to how structures refer to shifts in space 
    of the repeated background. These spacships can either: (1) collide, potentially destroying each other, completely changing the pattern; (2) collide and change one ship versus the 
    other; or (3) yield new ships. 


'''





In [20]:
############## 5.3 ########
'''
The goal of this exercise is to implement a Turing machine.
1. Read about Turing machines at http://thinkcomplex.com/tm.
2. Write a class called Turing that implements a Turing machine. For the
action table, use the rules for a 3-state busy beaver.
3. Write a class named TuringViewer that generates an image that represents the state of the tape and the position and state of the head. For
one example of what that might look like, see http://thinkcomplex.
com/turing.
'''


''' 
Attemtped this in matlab to give myself a fighting chance. Went to different resources of where to start for Matlab in generating a function, Ultmatly needed to create a fucntion that can handle
multiple diferrent rules to search for in the data. 
'''

 
### Tested author's code

'''
ultimately this functions as a one-dimentional turing test in one long tape that looks at each input until recieving some halt command' 
'''

    
    
# Solution

class Turing:
    """Represents a 1-D Turing machine."""

    def __init__(self, table, n, m=None):
        """Initializes the CA.

        tape: map from 
        n: number of rows
        m: number of columns

        Attributes:
        table:  rule dictionary that maps from triple to next state.
        array:  the numpy array that contains the data.
        next:   the index of the next empty row.
        """
        self.n = n
        self.m = n if m is None else m
        
        self.tape = np.zeros((n, self.m), dtype=np.uint8)
        self.head = np.zeros(n, dtype=np.int64)
        self.head[0] = m//2
        self.state = 'A'
        self.table = table
        self.next = 1

    def loop(self, steps=1):          
        """Executes the given number of time steps."""
        for i in range(steps):
            try:
                self.step()
            except StopIteration:
                break

    def step(self):
        """Executes one time step."""
        if self.state == 'HALT':
            raise StopIteration
            
        a = self.tape
        i = self.next
        a[i] = a[i-1]
        head = self.head[i-1]
        symbol = a[i-1, head]
        print(symbol, self.state, end=': ')
        new_symbol, move, self.state = self.table[symbol, self.state]
        print(new_symbol, move, self.state)
        
        a[i, head] = new_symbol
        if move == 'R':
            head += 1
        else:
            head -= 1
        self.head[i] = head
        self.next += 1
        
    def draw(self, start=0, end=None):
        """Draws the CA using pyplot.pcolor.
        
        start: index of the first column to be shown 
        end: index of the last column to be shown
        """
        # draw the cells
        a = self.tape[:, start:end]
        plt.imshow(a, cmap='Blues', alpha=0.4)
        
        # draw the read-write head
        xs = self.head
        ys = np.arange(len(xs))
        plt.plot(xs, ys, 'r.')

# Solution

# Here's the action table for a 3-state busy beaver.

table = {}
table[0, 'A'] = 1, 'R', 'B' 
table[0, 'B'] = 1, 'L', 'A' 
table[0, 'C'] = 1, 'L', 'B'
table[1, 'A'] = 1, 'L', 'C' 
table[1, 'B'] = 1, 'R', 'B' 
table[1, 'C'] = 1, 'R', 'HALT'

# Solution

# Make the Turning machine and run it

n = 15
m = 20
turing = Turing(table, n, m)

turing.loop(n-1)


0 A: 1 R B
0 B: 1 L A
1 A: 1 L C
0 C: 1 L B
0 B: 1 L A
0 A: 1 R B
1 B: 1 R B
1 B: 1 R B
1 B: 1 R B
1 B: 1 R B
0 B: 1 L A
1 A: 1 L C
1 C: 1 R HALT


In [ ]:
############## 5.4 ########
'''
This exercise asks you to implement and test several PRNGs.
For testing, you will need to install DieHarder, which you can download from
http://thinkcomplex.com/dh, or it might be available as a package for your
operating system.


Would have needed an Unbuntu environment to use DieHarder and could not find a package equivilent. 

However, Python uses the Mersenne Twister algorithm
''''





In [ ]:
############## 5.5  ########
'''
1. What is the demarcation problem?

    In essence, it is all about how the to differeniate between a true-real science, and a fake-psuedosience. Ultimately, what are the rules/conditions needed to be called a science or 
    cast away as a psuedosciecne -- usually beyond common sense as it could be used to identify how scientific is a field or even a hypothesis. 

2. How, according to Popper, does falsifiability solve the demarcation problem?

    This is the crux of something being label as scientific. Imagine that you said heroin was good for your health. This can be supported in either direction; you can test this notion. 
    However, if I were to say heroin was good for your health and any detriements to your health were actually masking the psotive effects, then this becomes significantly harder, if not 
    impossible to test. Falsifiability allows for scientists to experiement and observe evidence in either direction of their hypothesis--ultimately making and updating inferences abiout the real-world. 

3. Give an example of two theories, one considered scientific and one considered unscientific, that are successfully distinguished by the criterion
of falsifiability.

    Scientific: Behaviorism in psychology 

    Unscientific: Extrasensory Perception in the field of parapsychology 


4. Can you summarize one or more of the objections that philosophers and
historians of science have raised to Popper’s claim?

    One objection, by Lakatos, is that inductive methodology in which, while the university cannot be logically deduce, but rather inuctively uderstood through a series of observations. 
    Popper argues that science does not use induction with observations only falsifying hypotheses, never proving it. 

5. Do you get the sense that practicing philosophers think highly of Popper’s work?

    His ideas of falisibailty and the problem of induction seem to come unders scruitiny but still appears that he was well respected.
'''

